In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import chi2
import os
import sys

In [2]:
current_dir = os.getcwd() # Obtener la ruta del directorio actual del notebook
ROOT_PATH = os.path.dirname(current_dir) # Obtener la ruta del directorio superior
sys.path.insert(1, ROOT_PATH) # Insertar la ruta en sys.path

import root # Importar el módulo root

df_path = root.DIR_DATA_RAW + 'db_v0.xlsx'

In [3]:
df_raw = pd.read_excel(df_path)

In [4]:
df_processed = df_raw.copy()

## Eliminacion de registros sin informacion de mora o pago


Se eliminaran los registros que a la fecha de corte de la base de datos , no llegaron a la fecha de pago, por lo tal no se tiene informacion de si pagaran o estaran en mora, por lo que no es de utilidad para el entrenamiento del mododelo


In [5]:
df_processed = df_processed[df_processed['CAPITAL.1'] == 0]

In [6]:
df_processed.shape

(117654, 60)

## Generacion de columna de numero de prestamos que ha realizado la misma persona


Debido a que la columna 'ID_USUARIO' por si sola no es de utilidad para el modelo, antes de hacer la eliminacion de esta se utilizara para obtener informacion relevante, como el numero de veces que un mismo 'ID_USUARIO' ha solicitado un credito.


In [7]:
df_processed['NUM.CREDITOS SOLICITADOS'] = df_processed.groupby('ID_USUARIO').cumcount() + 1 

In [8]:
df_processed[['ID_USUARIO','NUM.CREDITOS SOLICITADOS']][df_processed['ID_USUARIO']==156]

,ID_USUARIO,NUM.CREDITOS SOLICITADOS
155,156,1
73026,156,2
83182,156,3
93316,156,4


## Generacion columna de usuario recurrente

In [9]:
def usuario_recurrente(df):
    if df['NUM.CREDITOS SOLICITADOS'] > 1:
        return 1
    else:
        return 0
    
df_processed['USUARIO RECURRENTE'] = df_processed.apply(usuario_recurrente, axis=1)
df_processed[['USUARIO RECURRENTE', 'NUM.CREDITOS SOLICITADOS']][df_processed['ID_USUARIO']==156]

,USUARIO RECURRENTE,NUM.CREDITOS SOLICITADOS
155,0,1
73026,1,2
83182,1,3
93316,1,4


## Eliminacion de columnas redundantes o innecesarias


Para decidir cuales columnas deben eliminarse, se tomo encuenta el conocimiento del negocio por parte de la empresa LUMON y lo sanalisi srealizados en el EDA (columnas sin varianza o alto numero de valores nulos)


In [10]:
columns_to_drop = ['ID_USUARIO', 'ID CREDITO', 'PRODUCTO', 'MONEDA', 'CUOTAS',
       'PERIODICIDAD CUOTAS','TASA CORRIENTE', 'TASA SEGURO',
       'TASA AVAL', 'IVA AVAL', 'DESC AVAL', 'DESC AVAL AL DESEMB','Cuotas pagadas',
       'GESTIÓN DIGITAL', 'DESC. X INCLUSION', 'IVA GEST DIG',
       'COD. PROMO DESC.', 'FACTURA VENTA','GESTIÓN DIGITAL.1', 'IVA',
       'VALOR DESEMBOLSADO', 'VALOR FUTURO', 'CAPITAL.1',
       'CAPITAL EN MORA', 'INT CORRIENTE.1', 'SEGURO.1', 'GESTIÓN DIGITAL.2',
       'IVA.1','GAC','Cuotas Futuras', 'ESTADO 2',
       'DEUDA A LA FECHA', 'DEUDA TOTAL CRÉDITO', 'SEGURO', 'INT MORA',
]

In [11]:
df_processed = df_processed.drop(columns_to_drop, axis=1)

In [12]:
df_processed.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'FECHA DESEMBOLSO',
       'PRÓXIMA FECHA PAGO', 'DÍAS MORA', 'Cuotas en mora', 'ESTADO 1',
       'TIPO EMPLEO', 'CIUDAD RESIDENCIA', 'TRABAJO', 'TIPO DE VIVIENDA',
       'ESTRATO', 'AÑOS EN LA VIVIENDA', 'INGRESOS MENSUALES',
       'GASTOS MENSUALES', 'INGRESOS ADICIONALES', 'TIPO DE CONTRATO',
       'PERIODO DE PAGO', 'ESTADO CIVIL', 'NIVEL EDUCATIVO',
       'PERSONAS A CARGO', 'NUMERO DE HIJOS', 'TIPO DE VEHICULO',
       'TIEMPO TRABAJO', 'NUM.CREDITOS SOLICITADOS', 'USUARIO RECURRENTE'],
      dtype='object')

## Divsion del data set


Como se observo en el EDA , los datos de la Informacion de los usuarios, tienen un 97% de datos nulos, por lo cual se decide dividir el data set, dejando uno que contenga todas las columnas pero solo con los registros que no tengan datos nulos en la Informacion de los usuarios, y otro dataset con todos los registros sin incluir las columnas de informacion de los usuarios.


In [13]:
df_users_info = df_processed.copy()
df_users_info = df_users_info.dropna(subset='AÑOS EN LA VIVIENDA')

In [14]:
df_users_info.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'FECHA DESEMBOLSO',
       'PRÓXIMA FECHA PAGO', 'DÍAS MORA', 'Cuotas en mora', 'ESTADO 1',
       'TIPO EMPLEO', 'CIUDAD RESIDENCIA', 'TRABAJO', 'TIPO DE VIVIENDA',
       'ESTRATO', 'AÑOS EN LA VIVIENDA', 'INGRESOS MENSUALES',
       'GASTOS MENSUALES', 'INGRESOS ADICIONALES', 'TIPO DE CONTRATO',
       'PERIODO DE PAGO', 'ESTADO CIVIL', 'NIVEL EDUCATIVO',
       'PERSONAS A CARGO', 'NUMERO DE HIJOS', 'TIPO DE VEHICULO',
       'TIEMPO TRABAJO', 'NUM.CREDITOS SOLICITADOS', 'USUARIO RECURRENTE'],
      dtype='object')

In [15]:
df_users_info.isnull().sum()

PLAZO                          0
CAPITAL                        0
INT CORRIENTE                  0
FECHA DESEMBOLSO               0
PRÓXIMA FECHA PAGO             0
DÍAS MORA                      0
Cuotas en mora                 0
ESTADO 1                       0
TIPO EMPLEO                   31
CIUDAD RESIDENCIA             45
TRABAJO                      133
TIPO DE VIVIENDA             789
ESTRATO                       14
AÑOS EN LA VIVIENDA            0
INGRESOS MENSUALES           708
GASTOS MENSUALES             728
INGRESOS ADICIONALES        1794
TIPO DE CONTRATO             737
PERIODO DE PAGO              742
ESTADO CIVIL                 795
NIVEL EDUCATIVO              803
PERSONAS A CARGO            1382
NUMERO DE HIJOS             1386
TIPO DE VEHICULO              19
TIEMPO TRABAJO               716
NUM.CREDITOS SOLICITADOS       0
USUARIO RECURRENTE             0
dtype: int64

In [16]:
df_users_info.shape

(4173, 27)

In [17]:
df_users_info[['DÍAS MORA','Cuotas en mora']][(df_users_info['DÍAS MORA']>0) & (df_users_info['Cuotas en mora'] == 0)].head()

,DÍAS MORA,Cuotas en mora
73,2617,0
81,2589,0
82,2579,0
84,2572,0
93,2531,0


In [18]:
df_users_info[['Cuotas en mora']][df_users_info['DÍAS MORA']>3].value_counts()

Cuotas en mora
1                 2630
0                 1543
Name: count, dtype: int64

### Varibale target 

In [19]:
df_users_info.loc[:, 'mora'] = (df_users_info['DÍAS MORA'] > 3).astype(int)

In [20]:
df_users_info[['DÍAS MORA','Cuotas en mora','mora','ESTADO 1']][(df_users_info['DÍAS MORA']>0) & (df_users_info['Cuotas en mora'] == 0)].head()

,DÍAS MORA,Cuotas en mora,mora,ESTADO 1
73,2617,0,1,PAGADO CON RETRASO
81,2589,0,1,PAGADO CON RETRASO
82,2579,0,1,PAGADO CON RETRASO
84,2572,0,1,PAGADO CON RETRASO
93,2531,0,1,PAGADO CON RETRASO


In [21]:
df_users_info['mora'].value_counts()

mora
1    4173
Name: count, dtype: int64

In [22]:
df_users_info.drop(['ESTADO 1','mora'], axis=1, inplace=True)

## DF credit info

In [23]:
users_info_columns = ['TIPO EMPLEO', 'CIUDAD RESIDENCIA', 'TRABAJO',
       'TIPO DE VIVIENDA', 'ESTRATO', 'AÑOS EN LA VIVIENDA',
       'INGRESOS MENSUALES', 'GASTOS MENSUALES', 'INGRESOS ADICIONALES',
       'TIPO DE CONTRATO', 'PERIODO DE PAGO', 'ESTADO CIVIL',
       'NIVEL EDUCATIVO', 'PERSONAS A CARGO', 'NUMERO DE HIJOS',
       'TIPO DE VEHICULO', 'TIEMPO TRABAJO']

df_reduced = df_processed.drop(users_info_columns,axis=1)

In [24]:
df_reduced.shape

(117654, 10)

### Variable target?

In [25]:
df_reduced[['DÍAS MORA','Cuotas en mora','ESTADO 1']][(df_reduced['DÍAS MORA']>0) & (df_reduced['Cuotas en mora'] == 0)].head()

,DÍAS MORA,Cuotas en mora,ESTADO 1
73,2617,0,PAGADO CON RETRASO
81,2589,0,PAGADO CON RETRASO
82,2579,0,PAGADO CON RETRASO
84,2572,0,PAGADO CON RETRASO
93,2531,0,PAGADO CON RETRASO


In [26]:
df_reduced[['Cuotas en mora']][df_reduced['DÍAS MORA']>3].value_counts()

Cuotas en mora
0                 32300
1                 11738
Name: count, dtype: int64

In [27]:
df_reduced.loc[:, 'mora'] = (df_reduced['DÍAS MORA'] > 3).astype(int)

In [28]:
df_reduced[['DÍAS MORA','Cuotas en mora','mora','ESTADO 1']][(df_reduced['DÍAS MORA']>0) & (df_reduced['Cuotas en mora'] == 0)].head()

,DÍAS MORA,Cuotas en mora,mora,ESTADO 1
73,2617,0,1,PAGADO CON RETRASO
81,2589,0,1,PAGADO CON RETRASO
82,2579,0,1,PAGADO CON RETRASO
84,2572,0,1,PAGADO CON RETRASO
93,2531,0,1,PAGADO CON RETRASO


In [29]:
df_reduced['mora'].value_counts()

mora
0    73616
1    44038
Name: count, dtype: int64

In [30]:
df_reduced.drop(['DÍAS MORA','Cuotas en mora','ESTADO 1','FECHA DESEMBOLSO'], axis=1, inplace=True)

### Guardar los datos en stage


In [31]:
ruta1 = root.DIR_DATA_RAW + 'db_raw_infousers.csv'
df_users_info.to_csv(ruta1, index=False) 

In [32]:
ruta2 = root.DIR_DATA_RAW + 'db_raw_reducida.csv'
df_reduced.to_csv(ruta2, index=False) 